<a href="https://colab.research.google.com/github/Wapiti08/GML/blob/main/bert_pra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install transformers
!pip3 install torch

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

In [ ]:
# define bert model name and max length for input sentence
model_name = 'bert-base-uncased'
max_length = 512

# initize the tokenzier with built-in bert tokenizer model
from transformers import BertTokenizerFast, BertForSequenceClassification, BertForTokenClassification
tokenzier = BertTokenizerFast.from_pretrained(model_name)

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a

In [ ]:
# prepare the data, need the texts and target --- include the train_test_split
data = fetch_20newsgroups(subset='all', remove=("headers", "footers", "quotes"))
texts = data.data
target_names = data.target_names
labels = data.target

train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.3)

In [ ]:
train_texts[:10]

['Q:   Who was consulted?  The Congress?  Industry?\n\nA:   We have on-going discussions with Congress and industry on\n     encryption issues, and expect those discussions to intensify\n     as we carry out our review of encryption policy.  We have\n     briefed members of Congress and industry leaders on the\n     decisions related to this initiative.\n\nCongress??  So we\'re playing politics before we talk to CPSR, academia,\nthe public, internet users?  I`ve heard of top-down design, but top-down \ndemocracy?? [Just whose state/district are the chip manufacturers in?]\n\nQ:   If the Administration were unable to find a technological\n     solution like the one proposed, would the Administration be\n     willing to use legal remedies to restrict access to more\n     powerful encryption devices?\n\nA:   This is a fundamental policy question which will be\n     considered during the broad policy review. [...bs about\n     the wonderfulness of the key escrow system...]\n\n     The Admi

In [ ]:
# tokenize the texts
token_encodings = tokenzier(train_texts, truncation=True, padding=True, max_length=max_length)

In [ ]:
# build the model with trainingarguments
model = BertForSequenceClassification.from_pretrained(model_name, num_labels = len(target_names))

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    num_train_epochs=10,
    per_device_train_batch_size=16,  # batch size per device during training
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    logging_steps=200,
    evaluation_strategy="steps",
    save_steps=200,
    output_dir='./'
    )


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19"
  },
  "initializer_range": 0.02,
  

In [ ]:
# trainer = new_trainer( model=model, args=training_args, train_dataset=train_texts)
trainer = Trainer( model=model, args=training_args, train_dataset=token_encodings)
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 10


ValueError: ignored